# Random Patch Sampler (3D)

In [1]:
# ms-python.python added
import os

In [2]:
import glob, shutil
import cv2
import json
import numpy as np
from skimage.io import imread, imsave, imshow
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from core.imageprep import dir_checker, random_crop, crop_generator, random_crop_batch, random_crop_batch_3D, patch_sampler_3D, volume_loader, sampler_3D
from datetime import datetime
from pprint import pprint
from tqdm.notebook import trange

%load_ext autoreload
%autoreload 2

In [3]:
# set parameters
# crop size
zdim = 20
ydim = 256
xdim = 256
crop_per_image = 200
seed = 100
timestamp = datetime.now().strftime("%Y_%m_%d_%H_%M")
output_folder = timestamp + '_' + str(crop_per_image) + 'x'
print("Crop Size z: {}".format(zdim))
print("Crop Size y: {}".format(ydim))
print("Crop Size x: {}".format(xdim))
print('Crop per Image: {}'.format(crop_per_image))
print('Seed: {}'.format(seed))
print('timestamp: {}'.format(timestamp))

Crop Size z: 20
Crop Size y: 256
Crop Size x: 256
Crop per Image: 200
Seed: 100
timestamp: 2020_01_27_13_31


In [4]:
# Prepare the training dataset
# Specify the input folder
# mainpath = '/Volumes/LaCie_DataStorage/PerlmutterData/dl_seg_project_raw'

def listdir_nohidden(dir):
    return [file for file in os.listdir(dir) if not file.startswith('.')]

working_folder = 'dl_seg_project_raw_3D_original'

mainpath = os.path.join('E:', 'PerlmutterData', working_folder)
img_seg_dir = 'data_Amira'
labeltypes = listdir_nohidden(os.path.join(mainpath, img_seg_dir))
ipdir = os.path.join(mainpath, img_seg_dir)
print(ipdir)
print(labeltypes)

E:PerlmutterData\dl_seg_project_raw_3D_original\data_Amira
['cell_membrane', 'lipid_droplets', 'mitochondria', 'nucleus']


In [5]:
# Create output folder
print("Check the output folder:")
dir_checker('data_crop_3D', mainpath)
dir_checker(output_folder, os.path.join(mainpath, 'data_crop_3D'))
opdir = os.path.join(mainpath, 'data_crop_3D', output_folder)
dir_checker('pars', opdir)
dir_checker('images', opdir)
dir_checker('labels', opdir)
print(opdir)

Check the output folder:
data_crop_3D exists in E:PerlmutterData\dl_seg_project_raw_3D_original
2020_01_27_13_31_200x does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D
pars does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x
images does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x
labels does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x
E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x


In [6]:
# create parameter
pars = dict(
            mainpath = mainpath,
            input_path = ipdir,
            output_path = opdir, 
            zdim = zdim,
            ydim = ydim,
            xdim = xdim, 
            crop_per_image = crop_per_image,
            )

with open(os.path.join(opdir, 'pars', 'pars_' + timestamp + '.json'), 'w') as outfile:
    json.dump(pars, outfile, indent=4)

In [7]:
data_path = [os.path.join(ipdir, labeltype) for labeltype in labeltypes ]
print("Path: {}".format(data_path))

Path: ['E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane', 'E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\lipid_droplets', 'E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\mitochondria', 'E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\nucleus']


In [8]:
# create list for filenames
ipimglist = {}
iplabellist = {}

for labeltype in labeltypes:
    imglist_tmp = glob.glob(os.path.join(ipdir, labeltype, 'images', '*'), recursive=True)
    labellist_tmp = glob.glob(os.path.join(ipdir, labeltype, 'labels', '*'), recursive=True)
    ipimglist[labeltype] = imglist_tmp
    iplabellist[labeltype] = labellist_tmp

In [9]:
pprint(ipimglist)

{'cell_membrane': ['E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane\\images\\data_d00_batch01_loc01_label01_stack01',
                   'E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane\\images\\data_d07_batch01_loc01_label01_stack01',
                   'E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane\\images\\data_d14_batch01_loc01_label01_stack01',
                   'E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane\\images\\data_d17_batch01_loc01_label01_stack01',
                   'E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane\\images\\data_d21_batch01_loc01_label01_stack01',
                   'E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\cell_membrane\\images\\data_d21_batch01_loc01_label01_stack02'],
 'lipid_droplets': ['E:PerlmutterData\\dl_seg_project_raw_3D_original\\data_Amira\\lipid_droplets\\images\\data_d00_batch01

In [10]:
for labeltype in labeltypes:
    tmp_imgcount_3D_list = listdir_nohidden(os.path.join(ipdir, labeltype, 'images'))
    tmp_labelcount_3D_list = listdir_nohidden(os.path.join(ipdir, labeltype, 'labels'))
    print(labeltype)
    print("Input Image Counts(3D): {}".format(len(tmp_imgcount_3D_list)))
    print("Input Label Counts(3D): {}".format(len(tmp_labelcount_3D_list)))

cell_membrane
Input Image Counts(3D): 6
Input Label Counts(3D): 6
lipid_droplets
Input Image Counts(3D): 5
Input Label Counts(3D): 5
mitochondria
Input Image Counts(3D): 5
Input Label Counts(3D): 5
nucleus
Input Image Counts(3D): 6
Input Label Counts(3D): 6


In [11]:
imgtypes = ['images', 'labels']
for imgtype in imgtypes:
    dir_checker(imgtype, opdir)
    for labeltype in labeltypes:
        dir_checker(labeltype, os.path.join(opdir, imgtype))

images exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x
cell_membrane does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images
lipid_droplets does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images
mitochondria does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images
nucleus does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images
labels exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x
cell_membrane does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels
lipid_droplets does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels
mitochondria does not exist in E:PerlmutterData\dl_seg_project_raw_3D_ori

In [12]:
# Batch Random Crop
for labeltype in labeltypes:     
    print("Croping the images from group {}...".format(labeltype))
    random_crop_batch_3D(ipimglist = ipimglist, 
                      iplabellist = iplabellist, 
                      opfolder = opdir, 
                      label
                         = labeltype, 
                      crop_size = [zdim, ydim, xdim], 
                      crop_per_image = crop_per_image, 
                      seed = seed)
    print("Finish")

Croping the images from group cell_membrane...
start random_crop_batch_3D



(20, 883, 2297)
0001 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0001 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0002 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0002 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0003 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0003 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0004 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0004 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27

0034 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0035 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0035 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0036 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0036 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0037 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0037 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0038 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\image

0068 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0069 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0069 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0070 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0070 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0071 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0071 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0072 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\image

0102 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0102 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0103 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0103 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0104 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0104 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\cell_membrane
0105 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\cell_membrane
0105 does not exist in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\label

...

0041 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0041 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0042 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0042 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0043 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0043 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0044 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0044 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0045 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200

0079 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0079 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0080 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0080 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0081 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0081 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0082 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0082 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0083 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200

0121 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0122 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0122 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0123 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0123 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0124 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0124 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0125 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0125 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200

0163 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0163 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0164 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0164 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0165 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0165 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0166 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\images\nucleus
0166 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200x\labels\nucleus
0167 exists in E:PerlmutterData\dl_seg_project_raw_3D_original\data_crop_3D\2020_01_27_13_31_200

Finish
